# Visits for DC2 Run 1.1

## What this notebook does: 
The Run 1.1 is supposed to have 750 WFD and 586 uDDF visits according to the google note. This is a selection of visits that overlap the region. For WFD this is simply the first 750, For uDDF we use the Twinkles cadence ie. get one visit per night per band subset of DDF, but at the lowest airmass.

The `write_visitlists_wfd(wfdDC2_fname, opsimout)` and `write_visitlists_uddf(uddfDC2_fname, opsimout)` will write out the csv files with no headers and no index columns with a whitespace separator
## To run this notebook, need :

### Code
- opsimsummary : https://github.com/rbiswas4/OpSimSummary

### Data Products: (see Survey Design Tracking )
- A list of WFD visits that overlap the protoDC2 region ('DC2VisitList_minion1016_v3_WFDvisits_nside512_WFDvisitsInuDD.csv')
- A list of uDDF visits that overlap the uDDF region ('DC2VisitList_minion1016_v3_DDvisits_nside512_DDvisitsInuDD.csv')



In [1]:
import opsimsummary as oss
import pandas as pd

In [2]:
print(oss.__version__)

1.3.7


In [3]:
wfdDC2_fname = 'DC2VisitList_minion1016_v3_WFDvisits_nside512_WFDvisitsInuDD.csv'
uddfDC2_fname = 'DC2VisitList_minion1016_v3_DDvisits_nside512_DDvisitsInuDD.csv'

In [4]:
NumVisitsWFD = 750 * 1.25 # Easy the first 750 visits . Padding for missing sensors due to small overlaps
NumVistitsuDDF = 586 * 19 * 1.25 + 150 # Needed some experimentation to get 586 visits per night 

In [5]:
opsimdb = '/Users/rbiswas/doc/projects/DESC/Notes/DC2_visits/minion_1016_desc_dithered_v4.db'

In [6]:
wfd = pd.read_csv(wfdDC2_fname, index_col='obsHistID')
uddf = pd.read_csv(uddfDC2_fname, index_col='obsHistID')

In [7]:
opsimout = oss.OpSimOutput.fromOpSimDB(opsimdb, subset='combined')

 reading from database sqlite:////Users/rbiswas/doc/projects/DESC/Notes/DC2_visits/minion_1016_desc_dithered_v4.db
SELECT * FROM Summary WHERE PROPID in (56, 54)


In [8]:
def write_visitlists_wfd(csvfname, opsimout, NumVisits=750, rootfname='../../scripts/protoDC2/protoDC2_visits_WFD',
                         header=False):
    wfd = pd.read_csv(csvfname, index_col='obsHistID')
    df = wfd.join(opsimout.summary).sort_values(by='expMJD').iloc[:NumVisits]
    groups = df.groupby('band')
    for band in groups.groups.keys():
        fname = rootfname + '_{}-band.txt'.format(band)
        groups.get_group(band).reset_index()[['obsHistID', 'expMJD']].to_csv(fname, index=False, sep=' ', header=header)



In [21]:
def write_visitlists_uddf(csvfname, opsimout, NumVisits=745*19 + 170, rootfname='../../scripts/protoDC2/protoDC2_visits_uDDF',
                         header=False):
    wfd = pd.read_csv(csvfname, index_col='obsHistID')
    df = wfd.join(opsimout.summary).sort_values(by='expMJD').iloc[:NumVisits]
    groups = df.reset_index().groupby('band')
    for band in groups.groups.keys():
        fname = rootfname + '_{}-band.txt'.format(band)
        x = groups.get_group(band).groupby('night')
        y = x[['obsHistID', 'expMJD', 'airmass']].agg(dict(airmass=min)).airmass
        #print(y.columns)
        y[['obsHistID', 'expMJD']].to_csv(fname, index=False, sep=' ', header=header)


In [22]:
write_visitlists_wfd(wfdDC2_fname, opsimout)

In [23]:
write_visitlists_uddf(uddfDC2_fname, opsimout)

In [24]:
!wc -l ../../scripts/protoDC2/protoDC2_visits_uDDF_*.txt

     117 ../../scripts/protoDC2/protoDC2_visits_uDDF_g-band.txt
     117 ../../scripts/protoDC2/protoDC2_visits_uDDF_i-band.txt
     117 ../../scripts/protoDC2/protoDC2_visits_uDDF_r-band.txt
     162 ../../scripts/protoDC2/protoDC2_visits_uDDF_u-band.txt
     115 ../../scripts/protoDC2/protoDC2_visits_uDDF_y-band.txt
     116 ../../scripts/protoDC2/protoDC2_visits_uDDF_z-band.txt
     744 total


In [25]:
!wc -l ../../scripts/protoDC2/protoDC2_visits_WFD_*-band.txt

      81 ../../scripts/protoDC2/protoDC2_visits_WFD_g-band.txt
     150 ../../scripts/protoDC2/protoDC2_visits_WFD_i-band.txt
     164 ../../scripts/protoDC2/protoDC2_visits_WFD_r-band.txt
      53 ../../scripts/protoDC2/protoDC2_visits_WFD_u-band.txt
     158 ../../scripts/protoDC2/protoDC2_visits_WFD_y-band.txt
     144 ../../scripts/protoDC2/protoDC2_visits_WFD_z-band.txt
     750 total


# Validate uDDF 

Since the number of uDDF visits is somewhat more difficult to see, let us validate it by a simpler selection where we take the first observation satisfying the constraints rather than looking for airmass to be the least. We can check that this gives the same numbers

In [10]:
df = uddf.join(opsimout.summary)

In [11]:
xx = df.query('propID==56').groupby(['night', 'band']).first().sort_values(by='expMJD').iloc[:745].groupby('filter').agg('count')

In [12]:
xx

,fID,sessionID,propID,fieldID,fieldRA,fieldDec,expDate,expMJD,visitTime,visitExpTime,...,wind,humidity,slewDist,slewTime,fiveSigmaDepth,ditheredRA,ditheredDec,descDitheredDec,descDitheredRA,descDitheredRotTelPos
filter,,,,,,,,,,,,,,,,,,,,,
g,117,117,117,117,117,117,117,117,117,117,...,117,117,117,117,117,117,117,117,117,117
i,117,117,117,117,117,117,117,117,117,117,...,117,117,117,117,117,117,117,117,117,117
r,117,117,117,117,117,117,117,117,117,117,...,117,117,117,117,117,117,117,117,117,117
u,163,163,163,163,163,163,163,163,163,163,...,163,163,163,163,163,163,163,163,163,163
y,115,115,115,115,115,115,115,115,115,115,...,115,115,115,115,115,115,115,115,115,115
z,116,116,116,116,116,116,116,116,116,116,...,116,116,116,116,116,116,116,116,116,116
